In [17]:
pip install --user ortools

In [18]:
import numpy as np
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform 

In [19]:
df = pd.read_csv("A-n33-k5.csv")

In [20]:
def merge(list1, list2):
      
    merged_list = [(list1[i], list2[i]) for i in range(0, len(list1))]
    return merged_list

In [21]:
def create_data_model():
    """Stores the data for the problem."""
    x = np.array(df['x'])
    y = np.array(df['y'])
    coordinates = merge(x, y)
    coordinates_list = [list(x) for x in coordinates]
    print(coordinates_list)
    coordinates_array = np.array(coordinates_list)
    dist_array = pdist(coordinates_array)
    dist_matrix = squareform(dist_array)
    demand = np.array(df['demand'])
    demand = demand.tolist()

    data = {}
    data['distance_matrix'] = dist_matrix
    data['demands'] = demand
    data['vehicle_capacities'] = [100, 100, 100, 100, 100]
    data['num_vehicles'] = 5
    data['depot'] = 0
    return data

In [22]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))

In [23]:
def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(demand_callback_index, 0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)

In [24]:
if __name__ == '__main__':
    main()

[[42, 68], [77, 97], [28, 64], [77, 39], [32, 33], [32, 8], [42, 92], [8, 3], [7, 14], [82, 17], [48, 13], [53, 82], [39, 27], [7, 24], [67, 98], [54, 52], [72, 43], [73, 3], [59, 77], [58, 97], [23, 43], [68, 98], [47, 62], [52, 72], [32, 88], [39, 7], [17, 8], [38, 7], [58, 74], [82, 67], [42, 7], [68, 82], [7, 48]]
Objective: 650
Route for vehicle 0:
 0 Load(0) ->  2 Load(23) ->  32 Load(26) ->  13 Load(49) ->  8 Load(59) ->  7 Load(78) ->  26 Load(80) ->  5 Load(88) ->  20 Load(96) ->  0 Load(96)
Distance of the route: 177m
Load of the route: 96

Route for vehicle 1:
 0 Load(0) ->  23 Load(14) ->  18 Load(27) ->  28 Load(42) ->  22 Load(61) ->  0 Load(61)
Distance of the route: 44m
Load of the route: 61

Route for vehicle 2:
 0 Load(0) ->  29 Load(8) ->  16 Load(18) ->  3 Load(32) ->  9 Load(50) ->  17 Load(74) ->  15 Load(92) ->  0 Load(92)
Distance of the route: 182m
Load of the route: 92

Route for vehicle 3:
 0 Load(0) ->  12 Load(9) ->  10 Load(29) ->  30 Load(49) ->  25 Load(